In [2]:
import geopandas as gpd
import matplotlib.pyplot as plt
import requests
import pandas as pd
import urllib.request, json
import ast
from shapely.geometry import box
from shapely.geometry import Point

In [2]:
def API_Request(Stations, SubscriptionKey):
    # API Headers
    url = f"https://gateway.apiportal.ns.nl/Spoorkaart-API/api/v1/traject.geojson?stations={Stations}"
    headers = {
        'Cache-Control': 'no-cache',
        'Ocp-Apim-Subscription-Key': SubscriptionKey,
    }

    # Make the API request
    try:
        req = urllib.request.Request(url, headers=headers)
        response = urllib.request.urlopen(req)
        response_content = response.read().decode('utf-8')

        # Load the response content into JSON
        geojson_data = json.loads(response_content)

        # Convert the GeoJSON data to a GeoDataFrame
        gdf = gpd.GeoDataFrame.from_features(geojson_data['features'])

        return gdf
    except urllib.error.HTTPError as e:
        print(f"HTTPError: {e.code} - {e.reason}")
        return None
    except Exception as e:
        print(f"Error: {e}")
        return None



In [3]:
# function to make a dictionary with the seats form the transervices csv
def SeatSorter(TrainServicesData, TrainTravelData):
    # Inladen CSV's met alle NS treindiensten en de reizigers data
    TrainServices = pd.read_csv(TrainServicesData, delimiter = ';')
    

    # Dictionary maken met per treinserie een dataframe met ieder stukje tussen stations er in
    dataframes_dict = {}

    TrainServices.loc[0,'Code']
    TrainServices.loc[0,'String'].split(',')

    for i in range(len(TrainServices)):
        a = TrainServices.loc[i,'String'].split(',')
        df = pd.DataFrame({'From':[],'To':[],'Seats':[]})
        for j in range(len(a) - 1):
            new_row = pd.DataFrame({'From': [a[j]], 'To': [a[j + 1]], 'Seats':[None]})
            df = pd.concat([df, new_row], ignore_index=True)
        dataframes_dict[TrainServices.loc[i,'Code']] = df

        a = TrainServices.loc[i,'String'].split(',')
        a = a[::-1]
        df = pd.DataFrame({'From':[],'To':[],'Seats':[]})
        for j in range(len(a) - 1):
            new_row = pd.DataFrame({'From': [a[j]], 'To': [a[j + 1]], 'Seats':[None]})
            df = pd.concat([df, new_row], ignore_index=True)
        dataframes_dict[TrainServices.loc[i,'Code'] + 1] = df

    # Dictionary vullen met stoel aantallen
    for i in range(len(TrainTravelData)):
        if TrainTravelData.loc[i, 'JourneyNumber'] % 2 != 0:
            a = TrainTravelData.loc[i, 'JourneyNumber']//100
            c = 0
            for j in range(len(dataframes_dict[a*100+ 1])):
                if TrainTravelData.loc[i, 'UserStopCodeBegin'].upper() == dataframes_dict[a*100+ 1].loc[j, 'From'].upper():
                    c = 1
                if c == 1:
                    if dataframes_dict[a*100 + 1].loc[j, 'Seats'] == None:
                        dataframes_dict[a*100+ 1].loc[j, 'Seats'] = TrainTravelData.loc[i, 'Seats']
                    else:
                        dataframes_dict[a*100+ 1].loc[j, 'Seats'] += TrainTravelData.loc[i, 'Seats']
                if TrainTravelData.loc[i, 'UserStopCodeEnd'].upper() == dataframes_dict[a*100+ 1].loc[j, 'To'].upper():
                    break

        else:
            a = TrainTravelData.loc[i, 'JourneyNumber']//100
            c = 0
            for j in range(len(dataframes_dict[a*100])):
                if TrainTravelData.loc[i, 'UserStopCodeBegin'].upper() == dataframes_dict[a*100].loc[j, 'From'].upper():
                    c = 1
                if c == 1:
                    if dataframes_dict[a*100].loc[j, 'Seats'] == None:
                        dataframes_dict[a*100].loc[j, 'Seats'] = TrainTravelData.loc[i, 'Seats']
                    else:
                        dataframes_dict[a*100].loc[j, 'Seats'] += TrainTravelData.loc[i, 'Seats']
                if TrainTravelData.loc[i, 'UserStopCodeEnd'].upper() == dataframes_dict[a*100].loc[j, 'To'].upper():
                    break

    return dataframes_dict

In [4]:
# function to assign a color to a number based on where in the range of numbers it lies
def interpolate_color(lower_limit, upper_limit, lower_color, upper_color, number):
    # Clamp the number between the lower and upper limit
    number = max(min(number, upper_limit), lower_limit)
    
    # Calculate the interpolation factor
    factor = (number - lower_limit) / (upper_limit - lower_limit)
    
    # Interpolate each RGB component
    interpolated_color = tuple(
        int(lower_component + (upper_component - lower_component) * factor)
        for lower_component, upper_component in zip(lower_color, upper_color)
    )
    
    #normalize the color for usage
    color = [interpolated_color / 255.0 for interpolated_color in interpolated_color] 

    return color

In [7]:

#   NS Bron aanpassen
# Inladen CSV en toevoegen kolommen voor Stoelen en Bezette stoelen
Keolis = pd.read_csv('Keolis.csv', delimiter= ';')
Arriva = pd.read_csv('Arriva.csv')
Qbuzz = pd.read_csv('Qbuzz.csv')
NS = pd.read_csv('OC_NS_20241007.csv')
TravelData = pd.concat([Keolis], ignore_index=True)
TravelData['Seats'] = 0
TravelData['Occupied Seats'] = 0

# Voor iedere treinserie het aantal stoelen toevoegen
for i in range(len(TravelData)):
    if TravelData.loc[i, 'VehicleType'] == "VIRM":
        TravelData.loc[i, 'Seats'] =  TravelData.loc[i, 'TotalNumberOfCoaches'] * 100
    if TravelData.loc[i, 'VehicleType'] == "DDZ":
        TravelData.loc[i, 'Seats'] =  TravelData.loc[i, 'TotalNumberOfCoaches'] * 100
    if TravelData.loc[i, 'VehicleType'] == "FLIRT FFF":
        TravelData.loc[i, 'Seats'] =  TravelData.loc[i, 'TotalNumberOfCoaches'] * 53
    if TravelData.loc[i, 'VehicleType'] == "ICM":
        TravelData.loc[i, 'Seats'] =  TravelData.loc[i, 'TotalNumberOfCoaches'] * 75
    if TravelData.loc[i, 'VehicleType'] == "ICNG25":
        TravelData.loc[i, 'Seats'] =  TravelData.loc[i, 'TotalNumberOfCoaches'] * 52
    if TravelData.loc[i, 'VehicleType'] == "SLT":
        TravelData.loc[i, 'Seats'] =  TravelData.loc[i, 'TotalNumberOfCoaches'] * 54
    if TravelData.loc[i, 'VehicleType'] == "SNG":
        TravelData.loc[i, 'Seats'] =  TravelData.loc[i, 'TotalNumberOfCoaches'] * 50
    if TravelData.loc[i, 'VehicleType'] == "SW7-25KV":
        TravelData.loc[i, 'Seats'] =  TravelData.loc[i, 'TotalNumberOfCoaches'] * 48
    if TravelData.loc[i, 'VehicleType'] == "SW9-25KV":
        TravelData.loc[i, 'Seats'] =  TravelData.loc[i, 'TotalNumberOfCoaches'] * 48
    if TravelData.loc[i, 'VehicleType'] == "GTW":
        TravelData.loc[i, 'Seats'] =  TravelData.loc[i, 'TotalNumberOfCoaches'] * 45
    if TravelData.loc[i, 'VehicleType'] == "Flirt":
        TravelData.loc[i, 'Seats'] =  TravelData.loc[i, 'TotalNumberOfCoaches'] * 57
    if TravelData.loc[i, 'VehicleType'] == "FLIRT":
        TravelData.loc[i, 'Seats'] =  TravelData.loc[i, 'TotalNumberOfCoaches'] * 57    
    if TravelData.loc[i, 'VehicleType'] == "Lint":
        TravelData.loc[i, 'Seats'] =  TravelData.loc[i, 'TotalNumberOfCoaches'] * 65
    if TravelData.loc[i, 'VehicleType'] == "WINK":
        TravelData.loc[i,'Seats'] = 153

# Maar 1 week behouden dus 3 dagen weghalen uit de tabel
TravelData['OperatingDay'] = pd.to_datetime(TravelData['OperatingDay'])
dates_to_exclude = pd.to_datetime(['2024-10-14', '2024-10-15', '2024-10-16'])
df_filtered = TravelData[~TravelData['OperatingDay'].isin(dates_to_exclude)]
TravelData = df_filtered.reset_index(drop=True)

#treincodes boven 700000 aanpassen
for i in range(len(TravelData)):
    if TravelData.loc[i, 'JourneyNumber'] > 700000:
        TravelData.loc[i, 'JourneyNumber'] -= 700000

#treincode tussen 200000 en 700000 aanpassen
for i in range(len(TravelData)):
    if 200000 < TravelData.loc[i, 'JourneyNumber'] < 700000:
        TravelData.loc[i, 'JourneyNumber'] -= 200000

display(TravelData)
TravelData.to_csv('TravelDataKeolis.csv')

,DataOwnerCode,OperatingDay,LinePlanningNumber,JourneyNumber,ReinforcementNumber,TimingLinkOrder,UserStopCodeBegin,UserStopCodeEnd,Occupancy,VehicleType,TotalNumberOfCoaches,Seats,Occupied Seats
0,Keo,2024-10-07,NaN,7909,0,1,ZL,NVD,2,FLIRT,4,228,0
1,Keo,2024-10-07,NaN,7911,0,1,ZL,NVD,2,FLIRT,3,171,0
2,Keo,2024-10-07,NaN,7913,0,1,ZL,NVD,2,FLIRT,4,228,0
3,Keo,2024-10-07,NaN,7915,0,1,ZL,NVD,2,FLIRT,4,228,0
4,Keo,2024-10-07,NaN,7916,0,1,NVD,ZL,2,FLIRT,4,228,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2501,Keo,2024-10-13,NaN,8586,0,1,KPN,ZL,2,FLIRT,4,228,0
2502,Keo,2024-10-13,NaN,8587,0,1,ZL,KPN,2,FLIRT,4,228,0
2503,Keo,2024-10-13,NaN,8588,0,1,KPN,ZL,2,FLIRT,4,228,0
2504,Keo,2024-10-13,NaN,8590,0,1,KPN,ZL,2,FLIRT,4,228,0


In [108]:
# Load the shapefile of the Netherlands
netherlands_shapefile_path = "ne_10m_admin_0_countries/ne_10m_admin_0_countries.shp"
world = gpd.read_file(netherlands_shapefile_path)
netherlands = world[world['NAME'] == "Netherlands"]

# Set the CRS for the Netherlands shapefile to WGS84 (assuming it uses lat/lon)
netherlands = netherlands.set_crs(epsg=4326)

# Reproject the Netherlands shapefile to UTM (Zone 31N for the Netherlands)
netherlands = netherlands.to_crs(epsg=32631)

# Set the CRS for the dataframe `df` (assuming df contains lat/lon coordinates)
if df.crs is None:
    df = df.set_crs(epsg=4326)  # Set to WGS84 if no CRS is defined

# Reproject `df` to the same UTM projection (EPSG:32631)
df = df.to_crs(epsg=32631)

# Define original limits in WGS84 (lat/lon)
lon_min, lon_max = 3, 7.5
lat_min, lat_max = 50, 54

# Create a GeoDataFrame to represent the bounding box
bbox = gpd.GeoDataFrame(
    {'geometry': [box(lon_min, lat_min, lon_max, lat_max)]}, 
    crs="EPSG:4326"
)

# Reproject the bounding box to UTM (to match the Netherlands and df projection)
bbox = bbox.to_crs(epsg=32631)

# Extract the limits from the reprojected bounding box
minx, miny, maxx, maxy = bbox.total_bounds

# Create figure
fig, ax = plt.subplots(figsize=[12, 8], dpi=600)

# Set the new limits using the reprojected UTM coordinates
ax.set_xlim([minx, maxx])
ax.set_ylim([miny, maxy])

# Plot the reprojected Netherlands
netherlands.plot(ax=ax, color='lightgray', edgecolor='gray', zorder=1)

# Plot your data points with associated colors
for i in range(len(df)):
    df.iloc[[i]].plot(ax=ax, color=ast.literal_eval(df.loc[i, 'color']), linewidth=1, zorder=2)

# Importeren steden
steden = pd.read_csv('Randstad-0.csv')

# Convert the 'steden' DataFrame to a GeoDataFrame
steden['geometry'] = steden.apply(lambda row: Point(row['Lng-coord'], row['Lat-coord']), axis=1)
steden_gdf = gpd.GeoDataFrame(steden, geometry='geometry')

# Set the CRS for the cities to WGS84 (assuming the coordinates are in lat/lon)
steden_gdf = steden_gdf.set_crs(epsg=4326)

# Reproject the cities to the same CRS as the map (UTM Zone 31N)
steden_gdf = steden_gdf.to_crs(epsg=32631)
steden_gdf2 = steden_gdf[(steden_gdf['Type'] == 'knooppuntIntercitystation') | (steden_gdf['Type'] == 'megastation') | (steden_gdf['Type'] == 'intercitystation')].reset_index(drop=True)
steden_gdf2.plot(ax=ax, color='green', marker='o', markersize=1, zorder=3)

plt.show()

In [101]:
#Sort seats according to the provided travel data en the known existing trainservices
TrainTravelData = pd.read_csv('TravelDataNS.csv', index_col=0)
TrainTravelData2 = TrainTravelData[TrainTravelData['OperatingDay'] == '2024-10-08'].reset_index(drop=True)
SortedSeats = SeatSorter('TrainServices.csv', TrainTravelData2)

In [102]:
#de dataframe met totale aantal stoelen per stukje route maken
df = pd.DataFrame({'From':[],'To':[],'Seats':[]})

for i in SortedSeats.keys():
    b = SortedSeats[i]
    d = 0
    c = 0
    for j in range(len(b)):
        if b.loc[j,'Seats'] == None:
            continue
        for k in range(len(df)):
            if b.loc[j,'From'].upper() == df.loc[k, 'From'].upper() and b.loc[j,'To'].upper() == df.loc[k, 'To'].upper():
                c = 1
                d = k
                break
            if b.loc[j,'To'].upper() == df.loc[k, 'From'].upper() and b.loc[j,'From'].upper() == df.loc[k, 'To'].upper():
                c = 1
                d = k
                break
        if c == 1:
            df.loc[d, 'Seats'] += b.loc[j,'Seats']
            c = 0
        else:
            new_row = pd.DataFrame({'From': [b.loc[j,'From'].upper()], 'To': [b.loc[j,'To'].upper()], 'Seats':[b.loc[j,'Seats']]})
            df = pd.concat([df, new_row], ignore_index=True)

In [103]:
#display(df)
#df.to_csv('SeatsPerTrajectoryTuesday.csv')

In [104]:
# Prompt for the SubscriptionKey before starting the loop
SubscriptionKey = input('Please enter your SubscriptionKey: ')

geometry_data = []

for i in range(len(df)):
    while True:
        try:
            # Make the API request
            geodata = API_Request(f'{df["From"][i]},{df["To"][i]}', SubscriptionKey)
            
            # If the API request is successful, add geometry data and break out of the loop
            if geodata is not None:
                geometry_data.append(geodata['geometry'].iloc[0])
                print(f'{i+1}/{len(df)}', end="\r")
                break
            else:
                # If the API fails, prompt for a new subscription key
                SubscriptionKey = input('Invalid SubscriptionKey. Please enter a new SubscriptionKey: ')
        except Exception as e:
            print(f"An error occurred: {e}")
            SubscriptionKey = input('Error occurred. Please enter a new SubscriptionKey: ')


In [105]:
#print(geometry_data)

In [106]:
df['geometry'] = geometry_data
df = gpd.GeoDataFrame(df, geometry='geometry')
#df.to_csv('SeatsPerTrajectoryTuesdayComplete.csv')

In [107]:
#volgorde zo aanpassen dat de hogere waardes altijd later worden geplot en dus over de lagere waardes heen plotten
df = df.sort_values(by='Seats', ascending=True).reset_index()

# kleuren assignen en toevoegen aan de geopanda
df['color'] = 0
for i in range(len(df)):
    df.loc[i, 'color'] = str(interpolate_color(df['Seats'].min(), df['Seats'].max(), (255,255,0), (255,0,0), df.loc[i, 'Seats']))


In [109]:
#display(df)
df.to_csv('PlotDataTuesday.csv')

,index,From,To,Seats,geometry,color
0,255,SD,ST,1000.0,"LINESTRING (657238.993 5783953.915, 657245.731...","[1.0, 1.0, 0.0]"
1,256,ST,STZ,1000.0,"LINESTRING (657965.094 5782833.783, 657973.926...","[1.0, 1.0, 0.0]"
2,254,BRN,SD,1000.0,"LINESTRING (655842.751 5786662.626, 655923.979...","[1.0, 1.0, 0.0]"
3,257,STZ,DLD,1000.0,"LINESTRING (657508.022 5781925.372, 657297.037...","[1.0, 1.0, 0.0]"
4,139,RSN,HON,19650.0,"LINESTRING (739952.735 5801592.99, 739864.815 ...","[1.0, 0.9529411764705882, 0.0]"
...,...,...,...,...,...,...
268,86,ASHD,ASB,261056.0,"LINESTRING (633634.17 5796040.942, 632996.071 ...","[1.0, 0.3764705882352941, 0.0]"
269,85,AC,ASHD,261056.0,"LINESTRING (634866.881 5793851.915, 634839.564...","[1.0, 0.3764705882352941, 0.0]"
270,90,ASDM,ASD,281267.0,"LINESTRING (631481.677 5803089.149, 631481.647...","[1.0, 0.3254901960784314, 0.0]"
271,80,UTVR,UT,302048.0,"LINESTRING (645394.058 5771945.629, 645341.792...","[1.0, 0.27450980392156865, 0.0]"
